In [1]:
from get_data_functions import func_initialise, fetch_fixture, fetch_player_stats
import pandas as pd
from pandasql import sqldf

In [2]:
func_initialise()

Fitzroy Package imported
tidyverse package imported
dplyr package imported


In [3]:
player_stats_2023_df = fetch_player_stats(source = 'afltables')

i Looking for data from 2023-01-01 to 2023-07-03
i fetching cached data from <github.com>
v fetching cached data from <github.com> ... done

i New data found for 9 matches
i Downloading data
v Downloading data ... done

i Processing XMLS
v Processing XMLS ... done



R[write to console]: Finished getting afltables data



In [15]:
player_stats_2023_df.columns.to_list()

['Season',
 'Round',
 'Date',
 'Local.start.time',
 'Venue',
 'Attendance',
 'Home.team',
 'HQ1G',
 'HQ1B',
 'HQ2G',
 'HQ2B',
 'HQ3G',
 'HQ3B',
 'HQ4G',
 'HQ4B',
 'Home.score',
 'Away.team',
 'AQ1G',
 'AQ1B',
 'AQ2G',
 'AQ2B',
 'AQ3G',
 'AQ3B',
 'AQ4G',
 'AQ4B',
 'Away.score',
 'First.name',
 'Surname',
 'ID',
 'Jumper.No.',
 'Playing.for',
 'Kicks',
 'Marks',
 'Handballs',
 'Goals',
 'Behinds',
 'Hit.Outs',
 'Tackles',
 'Rebounds',
 'Inside.50s',
 'Clearances',
 'Clangers',
 'Frees.For',
 'Frees.Against',
 'Brownlow.Votes',
 'Contested.Possessions',
 'Uncontested.Possessions',
 'Contested.Marks',
 'Marks.Inside.50',
 'One.Percenters',
 'Bounces',
 'Goal.Assists',
 'Time.on.Ground..',
 'Substitute',
 'Umpire.1',
 'Umpire.2',
 'Umpire.3',
 'Umpire.4',
 'group_id']

In [5]:
# Perform the aggregation and calculation
aggregated_df_FK_round = player_stats_2023_df.groupby(['team.name', 'round.roundNumber']).agg({'freesFor': 'sum', 'freesAgainst': 'sum'}).reset_index()
aggregated_df_FK_round['FK_diff'] = aggregated_df_FK_round['freesFor'] - aggregated_df_FK_round['freesAgainst']

# Print the resulting DataFrame
print(aggregated_df_FK_round[['team.name', 'round.roundNumber', 'FK_diff']].sort_values(ascending=False, by = 'FK_diff'))

         team.name  round.roundNumber  FK_diff
54     Collingwood                  2     19.0
60     Collingwood                  8     14.0
221       St Kilda                  7     14.0
80        Essendon                 13     14.0
227       St Kilda                 14     13.0
..             ...                ...      ...
213       Richmond                 14    -13.0
236   Sydney Swans                  8    -14.0
51         Carlton                 13    -14.0
191  Port Adelaide                  7    -14.0
186  Port Adelaide                  2    -19.0

[273 rows x 3 columns]


In [6]:
# Perform the aggregation and calculation
aggregated_df_FK_total = player_stats_2023_df.groupby(['team.name']).agg({'freesFor': 'sum', 'freesAgainst': 'sum'}).reset_index()
aggregated_df_FK_total['FK_diff'] = aggregated_df_FK_total['freesFor'] - aggregated_df_FK_total['freesAgainst']

# Print the resulting DataFrame
print(aggregated_df_FK_total[['team.name', 'FK_diff']].sort_values(ascending=False, by = 'FK_diff'))

            team.name  FK_diff
2      Brisbane Lions     50.0
15           St Kilda     42.0
6           Fremantle     37.0
4         Collingwood     33.0
3             Carlton     28.0
5            Essendon     16.0
18   Western Bulldogs     11.0
11          Melbourne      4.0
17  West Coast Eagles      2.0
0       NA_character_      0.0
8        Geelong Cats     -8.0
7          GWS Giants     -8.0
1      Adelaide Crows    -16.0
16       Sydney Swans    -17.0
9     Gold Coast Suns    -18.0
10           Hawthorn    -20.0
14           Richmond    -36.0
13      Port Adelaide    -41.0
12    North Melbourne    -45.0


In [7]:
# Filter the player_stats_2023_df DataFrame
filtered_df_home_FK_diff = player_stats_2023_df[player_stats_2023_df['team.name'] == player_stats_2023_df['home.team.name']]

# Perform the aggregation and calculation
aggregated_df_home_FK_total = filtered_df_home_FK_diff.groupby(['team.name']).agg({'freesFor': 'sum', 'freesAgainst': 'sum'}).reset_index()
aggregated_df_home_FK_total['FK_diff'] = aggregated_df_home_FK_total['freesFor'] - aggregated_df_home_FK_total['freesAgainst']

# Sort the DataFrame by FK_diff in descending order
sorted_df = aggregated_df_FK_total.sort_values(by='FK_diff', ascending=False)

# Print the resulting DataFrame
print(sorted_df[['team.name', 'FK_diff', 'freesFor', 'freesAgainst']])

            team.name  FK_diff  freesFor  freesAgainst
2      Brisbane Lions     50.0     293.0         243.0
15           St Kilda     42.0     274.0         232.0
6           Fremantle     37.0     287.0         250.0
4         Collingwood     33.0     296.0         263.0
3             Carlton     28.0     284.0         256.0
5            Essendon     16.0     252.0         236.0
18   Western Bulldogs     11.0     265.0         254.0
11          Melbourne      4.0     261.0         257.0
17  West Coast Eagles      2.0     264.0         262.0
0       NA_character_      0.0       0.0           0.0
8        Geelong Cats     -8.0     267.0         275.0
7          GWS Giants     -8.0     206.0         214.0
1      Adelaide Crows    -16.0     298.0         314.0
16       Sydney Swans    -17.0     277.0         294.0
9     Gold Coast Suns    -18.0     264.0         282.0
10           Hawthorn    -20.0     228.0         248.0
14           Richmond    -36.0     240.0         276.0
13      Po

In [8]:
# Perform the aggregation and calculation
aggregated_df_FK_total_sum = player_stats_2023_df.groupby(['team.name']).agg({'freesFor': 'sum', 'freesAgainst': 'sum'}).reset_index()

# Print the resulting DataFrame
print(aggregated_df_FK_total[['team.name', 'freesFor', 'freesAgainst']].sort_values(ascending=False, by = 'freesFor'))

            team.name  freesFor  freesAgainst
1      Adelaide Crows     298.0         314.0
4         Collingwood     296.0         263.0
2      Brisbane Lions     293.0         243.0
6           Fremantle     287.0         250.0
13      Port Adelaide     285.0         326.0
3             Carlton     284.0         256.0
16       Sydney Swans     277.0         294.0
15           St Kilda     274.0         232.0
8        Geelong Cats     267.0         275.0
18   Western Bulldogs     265.0         254.0
17  West Coast Eagles     264.0         262.0
9     Gold Coast Suns     264.0         282.0
11          Melbourne     261.0         257.0
12    North Melbourne     255.0         300.0
5            Essendon     252.0         236.0
14           Richmond     240.0         276.0
10           Hawthorn     228.0         248.0
7          GWS Giants     206.0         214.0
0       NA_character_       0.0           0.0


In [15]:
# Perform the aggregation and calculation
aggregated_df_behinds_goal_diff = player_stats_2023_df.groupby(['team.name','round.roundNumber']).agg({'goals': 'sum', 'behinds': 'sum'}).reset_index()
aggregated_df_behinds_goal_diff['behinds_goals_diff'] = aggregated_df_behinds_goal_diff['behinds'] - aggregated_df_behinds_goal_diff['goals']
aggregated_df_behinds_goal_diff.sort_values(by= 'behinds_goals_diff', ascending=False)

,team.name,round.roundNumber,goals,behinds,behinds_goals_diff
243,Sydney Swans,16,6.0,16.0,10.0
143,Hawthorn,2,4.0,13.0,9.0
71,Essendon,4,11.0,19.0,8.0
15,Adelaide Crows,7,7.0,13.0,6.0
51,Carlton,13,6.0,12.0,6.0
...,...,...,...,...,...
54,Collingwood,2,21.0,7.0,-14.0
151,Hawthorn,10,22.0,7.0,-15.0
21,Adelaide Crows,13,27.0,12.0,-15.0
104,GWS Giants,7,17.0,2.0,-15.0
